## Question 1

* Install `uv`
* What's the version of uv you installed?
* Use `--version` to find out

In [1]:
uv --version

uv 0.9.5


Note: you may need to restart the kernel to use updated packages.


## Initialize an empty uv project

In [4]:
uv init

Initialized project `homework`
Note: you may need to restart the kernel to use updated packages.


## Question 2

* Use uv to install Scikit-Learn version 1.6.1 
* What's the first hash for Scikit-Learn you get in the lock file?
* Include the entire string starting with sha256:, don't include quotes

In [2]:
uv add "scikit-learn==1.6.1"

Using CPython 3.12.1 interpreter at: /home/codespace/.python/current/bin/python
Creating virtual environment at: .venv
Resolved 6 packages in 464ms                                         
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/12.54 MiB           
⠙ Preparing packages... (0/1)-------------- 14.90 KiB/12.54 MiB         
⠙ Preparing packages... (0/1)-------------- 30.90 KiB/12.54 MiB         
⠙ Preparing packages... (0/1)-------------- 46.90 KiB/12.54 MiB         
⠙ Preparing packages... (0/1)-------------- 62.90 KiB/12.54 MiB         
⠙ Preparing packages... (0/1)-------------- 78.90 KiB/12.54 MiB         
⠙ Preparing packages... (0/1)-------------- 94.90 KiB/12.54 MiB         
⠙ Preparing packages... (0/1)-------------- 110.90 KiB/12.54 MiB        
⠙ Preparing packages... (0/1)-------------- 126.90 KiB/12.54 MiB        
⠙ Preparing packages... (0/1)-------------- 142.90 KiB/12.54 MiB        


The first hash for Scikit-Learn you get in uv.lock: `sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e`


## Models

We have prepared a pipeline with a dictionary vectorizer and a model.

It was trained (roughly) using this code:

```
categorical = ['lead_source']
numeric = ['number_of_courses_viewed', 'annual_income']

df[categorical] = df[categorical].fillna('NA')
df[numeric] = df[numeric].fillna(0)

train_dict = df[categorical + numeric].to_dict(orient='records')

pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression(solver='liblinear')
)

pipeline.fit(train_dict, y_train)
```

> **Note**: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download it [here](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/cohorts/2025/05-deployment/pipeline_v1.bin).

With `wget`:

```
wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
```

In [3]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-27 17:27:18--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-27 17:27:19--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin’

pipeline_v1.bin


## Question 3

Let's use the model!

* Write a script for loading the pipeline with pickle
* Score this record:

```json
{
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}
```

What's the probability that this lead will convert? 

* 0.333
* 0.533
* 0.733
* 0.933

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum pipeline_v1.bin
7d17d2e4dfbaf1e408e1a62e6e880d49 *pipeline_v1.bin
```